# Prompt Engineering

## Setup

Before we can start having fun, let's setup the key libraries and functions we'll need. These will allow us to:
- Call OpenAI's API
- Call Bing's API

In [ ]:
%pip install python-dotenv
%pip install pprintpp
%pip install openai

## Environment Variables

Create an `.env` file in the root directory of this project. It should contain the following variables:

```env
BING_SEARCH_V7_SUBSCRIPTION_KEY="<your key here>"
BING_SEARCH_V7_ENDPOINT="https://api.bing.microsoft.com/"
OPENAI_KEY="<your openai key>"
OPENAI_AZURE_KEY="<your azure openai key>"
OPENAI_AZURE_BASE_URL="<your azure openai endpoint>"
OPENAI_AZURE_API_VERSION="2023-05-15"
OPENAI_AZURE_CHAT_COMPLETION_DEPLOYMENT_NAME="<your azure openai deployment name for gpt 3.5 or gpt 4>"
```

In [12]:
# Load environment variables
import os
from pprint import pprint
import requests
from dotenv import load_dotenv
load_dotenv()

True

## Create helper functions
These are helper functions that will allow us to call the APIs and get the results we need.


1. Search for content using Bing Search using `bing_search`

In [13]:
# This function returns search results via Bing Search API
def bing_search(query):
    # Add your Bing Search V7 subscription key and endpoint to your environment variables.
    subscription_key = os.environ["BING_SEARCH_V7_SUBSCRIPTION_KEY"]    
    endpoint = os.environ["BING_SEARCH_V7_ENDPOINT"] 

    # Ensure that the subscription key & endpoint have been set, else throw an error.
    if not subscription_key or not endpoint:
        raise Exception("Set your environment variables for your subscription key and endpoint.")

    # Add a trailing slash to the endpoint if needed
    if endpoint[-1] != "/":
        endpoint = endpoint + "/"
    endpoint = endpoint + "v7.0/search"


    # Construct a request
    mkt = "en-US"
    params = { "q": query, "mkt": mkt }
    headers = { "Ocp-Apim-Subscription-Key": subscription_key }

    # Call the API
    try:
        response = requests.get(endpoint, headers=headers, params=params)
        response.raise_for_status()

        search_results = response.json()
    except Exception as ex:
        raise ex
    
    return search_results

In [14]:
# Perform a test search for the word "Microsoft"
result = bing_search("Microsoft")
# Print the first 2 results within the JSON response.
pprint(result["webPages"]["value"][0:2], indent=4, width=100, depth=2)

[   {   'dateLastCrawled': '2023-06-26T02:04:00.0000000Z',
        'deepLinks': [...],
        'displayUrl': 'https://www.microsoft.com/en-us',
        'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.0',
        'isFamilyFriendly': True,
        'isNavigational': True,
        'language': 'en',
        'name': 'Microsoft – Cloud, Computers, Apps & Gaming',
        'snippet': 'Explore Microsoft products and services for your home or business. Shop '
                   'Surface, Microsoft 365, Xbox, Windows, Azure, and more. Find downloads and get '
                   'support.',
        'url': 'https://www.microsoft.com/en-us/'},
    {   'dateLastCrawled': '2023-06-25T15:14:00.0000000Z',
        'displayUrl': 'https://support.microsoft.com',
        'id': 'https://api.bing.microsoft.com/api/v7/#WebPages.1',
        'isFamilyFriendly': True,
        'isNavigational': False,
        'language': 'en',
        'name': 'Microsoft Support',
        'searchTags': [...],
        'snippet

2. Pretty Prining functions:

    - `bold` - Print text in bold
    - `wrap` - Wraps text with a default width of 100 characters.

In [66]:
import textwrap

def bold(text):
    print(f'\033[1m{text}\033[0m')

# Testing the bold function
bold("Testing the bold functionality.")

def italic(text):
    print(f'\033[3m{text}\033[0m')

# Testing the italic function
italic("Testing the italic functionality.")

def wrap(text, line_length=100):
    for line in text.split('\n'):
        wrapper = textwrap.TextWrapper(width=line_length)
        word_list = wrapper.wrap(text=line)

        # Print each line
        for l in word_list:
            print(l)

# Testing the wrap function
wrap("The quick brown fox jumped over the lazy dog.", 10)

def box(text):    
    bold(f'\n\n┌{"─" * len(text)}┐')
    bold(f'│{text}│')
    bold(f'└{"─" * len(text)}┘')

# Testing the box function
box("Heading")

Testing the bold functionality.
Testing the italic functionality.
The quick
brown fox
jumped
over the
lazy dog.


┌───────┐
│Heading│
└───────┘


3. Message helper functions: OpenAI's Chat Completions expects messages in the following format:
    ```python
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"}
    ]
    ```
    To assist with this, we have the following functions:
    
    - `validate_message` - Validates that a message is in the expected format.
    - `print_messages` - Prints messages.

In [16]:
def validate_messages(messages):
    # Define the acceptable roles
    valid_roles = ["system", "user", "assistant"]

    # Check if messages is a list
    if not isinstance(messages, list):
        raise TypeError("Input should be a list")

    # Iterate over the list of messages
    for message in messages:
        # Check if message is a dictionary
        if not isinstance(message, dict):
            raise TypeError("Each message should be a dictionary")

        # Check if the necessary keys exist in the dictionary
        if "role" not in message or "content" not in message:
            raise ValueError("Each message should have 'role' and 'content' keys")

        # Check if role is valid
        if message["role"] not in valid_roles:
            raise ValueError(f"Role should be one of {valid_roles}")

    # If all messages pass the checks, return True
    return True

# Test valid messages
if validate_messages([
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Knock knock."},
    {"role": "assistant", "content": "Who's there?"}
]):
    print("Messages are valid!")

# Test invalid messages
try:
    validate_messages([
        "You are a helpful assistant.",
    ])
except Exception as ex:    
    print("Messages are invalid because:")
    print(ex)

def print_messages(messages):
    # Validate the messages first
    if validate_messages(messages):
        # Iterate over the messages and print each one in the desired format
        for message in messages:                        
            bold(f'\n[{message["role"]}]:')
            wrap(message["content"])

# Test the print_messages function
print_messages([
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Knock knock."},
    {"role": "assistant", "content": "Who's there?"}
])

Messages are valid!
Messages are invalid because:
Each message should be a dictionary

[system]:
You are a helpful assistant.

[user]:
Knock knock.

[assistant]:
Who's there?


4. Helper message to call the ChatCompletions endpoint (direct or via azure) using `perform_chat_completion`. 
    ***Note:*** Can call azure OR openai directly based on the type parameter. Defaults to openai.

    This function has the following parameters:

    | Parameter | Description | Is Required | Default Value |
    | --- | --- | --- | --- |
    | `messages` | The messages to send to the API. | Yes | N/A |
    | `type` | The type of API to call. Can be `openai` or `azure`. | No | `openai` |
    | `temperature` | What temperature to use for the API call. A value between 0 and 2 | No | `0` |
    | `top_p` | What top_p to use for the API call. A value between 0 and 1 | No | `1` |
    | `max_tokens` | What max_tokens to use for the API call. A value between 1 and 4096 | No | `2048` |

In [17]:
import sys

# This function calls the OpenAI (or Azure OpenAI)'s completion endpoint
def perform_chat_completion(messages, type="openai", temperature=0, top_p=0.5, max_tokens=2048):
    # perform checks else raise exception
    if type != 'openai' and type != 'azure':
        raise Exception("type must be either openai or azure")
    
    validate_messages(messages)
    
    if temperature < 0 or temperature > 2:
        raise Exception("temperature must be between 0 and 2")
    
    if top_p < 0 or top_p > 1:
        raise Exception("top_p must be between 0 and 1")

    if max_tokens < 1 or max_tokens > 4096:
        raise Exception("max_tokens must be between 1 and 2048")


    # check if type is openai or azure
    if type == "openai":        
        if not os.environ["OPENAI_KEY"]:
            raise Exception("Set your OPENAI_KEY environment variable.")
        
        import openai        
        openai.api_type = "openai"
        openai.api_version = None
        openai.api_base = "https://api.openai.com/v1/"
        openai.api_key = os.environ["OPENAI_KEY"]
            

        try:
            results = openai.ChatCompletion.create(
                model='gpt-3.5-turbo',
                messages=messages,
                temperature=temperature,
                top_p=top_p,
                max_tokens=max_tokens
            )
        except Exception as ex:
            raise ex
    else:
        if not os.environ["OPENAI_AZURE_KEY"]:
            raise Exception("Set your OPENAI_AZURE_KEY environment variable.")
        if not os.environ["OPENAI_AZURE_BASE_URL"]:
            raise Exception("Set your OPENAI_AZURE_BASE_URL environment variable.")
        if not os.environ["OPENAI_AZURE_API_VERSION"]:
            raise Exception("Set your OPENAI_AZURE_API_VERSION environment variable.")
        if not os.environ["OPENAI_AZURE_CHAT_COMPLETION_DEPLOYMENT_NAME"]:
            raise Exception("Set your OPENAI_AZURE_CHAT_COMPLETION_DEPLOYMENT_NAME environment variable.")
                
        import openai
        openai.api_type = 'azure'
        openai.api_version = os.environ["OPENAI_AZURE_API_VERSION"]
        openai.api_base = os.environ["OPENAI_AZURE_BASE_URL"]
        openai.api_key = os.environ["OPENAI_AZURE_KEY"]
        
        try:
            results = openai.ChatCompletion.create(
                deployment_id=os.environ["OPENAI_AZURE_CHAT_COMPLETION_DEPLOYMENT_NAME"],
                messages=messages,
                temperature=temperature,
                top_p=top_p,
                max_tokens=max_tokens
            )
        except Exception as ex:
            raise ex
        
    return results        


In [18]:
# Note: You can skip this step if you are using Azure OpenAI Service
box("Call Chat Completions endoint using OpenAI")
completion_message_request = [
   {"role": "system", "content": "You are a helpful assistant."},
   {"role": "user", "content": "Hi there."}
]
print_messages(completion_message_request)
chat_completion = perform_chat_completion(completion_message_request, "openai")
print_messages([chat_completion["choices"][0]["message"]])






┌──────────────────────────────────────────┐
│Call Chat Completions endoint using OpenAI│
└──────────────────────────────────────────┘

[system]:
You are a helpful assistant.

[user]:
Hi there.

[assistant]:
Hello! How can I assist you today?


In [19]:
# Note: You can skip this step if you are using OpenAI's API
box("Call Chat Completions endoint using  Azure")
completion_message_request = [
   {"role": "system", "content": "You are a helpful assistant."},
   {"role": "user", "content": "Hi there."}
]
print_messages(completion_message_request)
chat_completion = perform_chat_completion(completion_message_request, "azure")
print_messages([chat_completion["choices"][0]["message"]])



┌──────────────────────────────────────────┐
│Call Chat Completions endoint using  Azure│
└──────────────────────────────────────────┘

[system]:
You are a helpful assistant.

[user]:
Hi there.

[assistant]:
Hello! How can I assist you today?


## Prompt Engineering 

### Example 1: Simple conversational scenarios

The following code snippet shows the most basic way to use the ChatGPT and GPT-4 models with the Chat Completion API. 

In this example we ask the model to complete the following conversation:
```log
system: You are a helpful assistant.
user: Who were the founders of Microsoft?
assistant:
```

In [25]:
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
prompt_messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who were the founders of Microsoft?"}
]
completion_response = perform_chat_completion(prompt_messages, type)
box("Example 1: Simple conversational scenarios")
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌──────────────────────────────────────────┐
│Example 1: Simple conversational scenarios│
└──────────────────────────────────────────┘

[system]:
You are a helpful assistant.

[user]:
Who were the founders of Microsoft?

[assistant]:
The founders of Microsoft are Bill Gates and Paul Allen.


As you can see in the code above, the chat completion API expects messages to be in the following format:   
```json
[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Knock knock."},
    {"role": "assistant", "content": "Who's there?"},
    {"role": "user", "content": "Orange."},
]
```

Depending on the role you should use the following values:
| Role | Purpose | Explaination | Example |
| --- | --- | --- | --- |
| `system` | Provide some context and/or instructions to the model. | The system message is included at the beginning of the prompt and is used to prime the model and you can include a variety of information in the system message including: <br> - A brief description of the assistant <br> - The personality of the assistant <br> - Instructions for the assistant <br> - Data or information needed for the model | `{"role": "system", "content": "You are a helpful assistant."}` |
| `user` | A question/message for the model to actually respond to. | After the system message, you can include a series of messages between the user and the assistant. You denote who the message is from by setting the role to user or assistant. | `{"role": "user", "content": "Knock knock."}` |



**⚠️ You might be wondering how this is different from the previous models.**

Unlike previous GPT-3 models, the ChatGPT model is specifically designed to be a conversational interface. The conversational nature of the model makes it easier to interact with and to take advantage of the full power of its capabilities.

Previous models were text-in and text-out (i.e., they accepted a prompt string and returned a completion to append to the prompt).

The ChatGPT model is conversation-in and message-out. (i.e., it expects a prompt string that is formatted in a specific chat-like transcript format and returns a completion that represents a model-written message in the chat)


### Example 2: Non conversational scenarios

While the Chat Completion API is optimized to work with multi-turn conversations, it also can be used for non chat scenarios. 

This example is set as follows:
| Role | Explanation |
| --- | --- | 
| `system` | You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a book and you will respond with a short summary. Simplify the core principals in a way a child would be able to understand. | 
| `user` | Thinking, Fast and Slow by Daniel Kahneman |



In [26]:
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
prompt_messages = [
    {"role": "system", "content": "You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a book and you will respond with a short summary. Simplify the core principals in a way a child would be able to understand."},
    {"role": "user", "content": "Thinking, Fast and Slow by Daniel Kahneman"}
]
completion_response = perform_chat_completion(prompt_messages, type)
box("Example 2: Non conversational scenarios")
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌───────────────────────────────────────┐
│Example 2: Non conversational scenarios│
└───────────────────────────────────────┘

[system]:
You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a
book and you will respond with a short summary. Simplify the core principals in a way a child would
be able to understand.

[user]:
Thinking, Fast and Slow by Daniel Kahneman

[assistant]:
"Thinking, Fast and Slow" is a book that talks about how our brain works in two different ways. One
way is fast and automatic, like when we catch a ball without thinking. The other way is slow and
deliberate, like when we solve a math problem. The book explains how these two ways of thinking can
sometimes make us make mistakes, and how we can learn to make better decisions by being aware of
them.


# Example 3: Role Prompting

Here lets explore the `Be specific` and `Be descriptive` best practices listed at https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/prompt-engineering#best-practices.

- `Be Specific`: Leave as little to interpretation as possible. Restrict the operational space.
- `Be Descriptive`: Use analogies.

In this example we will show how being specific and descriptive can help us get better results. 

**Scenario 3.1: Basic prompt**

| Role | Explanation |
| --- | --- |
| `user` | What is 1000*1000/20*50? |

**Scenario 3.2: Better prompt using Role Prompting**

| Role | Explanation |
| --- | --- |
| `system` | Assume you are a math teacher and you are going to teach Order Of Operations Activities (BODMAS, BIMDAS, PEMDAS, GEMS & More) to your students. |
| `user` | What is 1000*1000/20*50? |


In [35]:
box("Scenario 3.1: Basic prompt")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
prompt_messages = [    
    {"role": "user", "content": "What is 1000*1000/20*50?"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌──────────────────────────┐
│Scenario 3.1: Basic prompt│
└──────────────────────────┘

[user]:
What is 1000*1000/20*50?

[assistant]:
1000*1000/20*50 = 50,000


In [37]:
box("Scenario 3.2: Better prompt using Role Prompting")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
prompt_messages = [    
    {"role": "user", "content": "Assume you are a math teacher and you are going to teach PEMDAS (Parentheses, Exponents, Multiplication and Division from left to right, and Addition and Subtraction from left to right) to your students. What is 1000*1000/20*50?"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌────────────────────────────────────────────────┐
│Scenario 3.2: Better prompt using Role Prompting│
└────────────────────────────────────────────────┘

[user]:
Assume you are a math teacher and you are going to teach PEMDAS (Parentheses, Exponents,
Multiplication and Division from left to right, and Addition and Subtraction from left to right) to
your students. What is 1000*1000/20*50?

[assistant]:
To solve this problem using PEMDAS, we need to follow the order of operations:
1. Parentheses (none)
2. Exponents (none)
3. Multiplication and Division from left to right:
1000 * 1000 = 1,000,000
1,000,000 / 20 = 50,000
50,000 * 50 = 2,500,000
4. Addition and Subtraction from left to right (none)
Therefore, 1000*1000/20*50 = 2,500,000.


### Example 4: Providing instructions to get expected results

To ensure the model provides the expected results, you can provide instructions to the model within the system message.

In this example, show the difference beteen providing instructions and not providing instructions. We will use the same prompt for both examples, but the system message will be different.

**Scenario 4.1: No instructions**

| Role | Explanation |
| --- | --- |
| `system` | You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a book and you will respond with a short summary. Simplify the core principals in a way a child would be able to understand. |
| `user` | How to build your Technology Strategy by Rohit Lakhanpal |

**Scenario 4.2: With instructions**

| Role | Explanation |
| --- | --- |
| `system` | You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a book and you will respond with a short summary. Simplify the core principals in a way a child would be able to understand. <br> <br> **Instructions:** <br> - If you don't know about the book, say "I don't know". <br> - Don't make up information. |
| `user` | How to build your Technology Strategy by Rohit Lakhanpal |


**Note**: No book called "How to build your Technology Strategy" by "Rohit Lakhanpal" exists. The model will make up information if you don't provide instructions. 

In [39]:
box("Scenario 4.1: No instructions")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
system_message = "You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a book and you will respond with a short summary. Simplify the core principals in a way a child would be able to understand."
book = "How to build your Technology Strategy by Rohit Lakhanpal"
prompt_messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": book}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌─────────────────────────────┐
│Scenario 4.1: No instructions│
└─────────────────────────────┘

[system]:
You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a
book and you will respond with a short summary. Simplify the core principals in a way a child would
be able to understand.

[user]:
How to build your Technology Strategy by Rohit Lakhanpal

[assistant]:
This book is about making a plan for using technology in a smart way. It helps you think about what
you want to achieve with technology and how to make it happen. It's like making a map to get to your
destination, but for technology!


In [38]:
box("Scenario 4.2: With instructions")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly
system_message = """You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a book and you will respond with a short summary. Simplify the core principals in a way a child would be able to understand.

#Instructions:
- If you don't know about the book, say "I don't know". 
- Don't make up information.
"""
prompt_messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": "How to be a Tech Stragist by Rohit Lakhanpal"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌───────────────────────────────┐
│Scenario 4.2: With instructions│
└───────────────────────────────┘

[system]:
You are an assistant designed to summarise this non-fiction books. Users will paste in a name of a
book and you will respond with a short summary. Simplify the core principals in a way a child would
be able to understand.
#Instructions:
- If you don't know about the book, say "I don't know".
- Don't make up information.

[user]:
How to be a Tech Stragist by Rohit Lakhanpal

[assistant]:
I don't know.


As seen in example above, when you don't provide instructions, the model will make up information. This is known as `hallucination`. To combat this, we used the system message to provide instructions to the model. This is part of the best practice known as `Giving the model an out`.

**Give the model an out**: It can sometimes be helpful to give the model an alternative path if it is unable to complete the assigned task. For example, when asking a question over a piece of text you might include something like `respond with 'not found' if the answer is not present`. This can help the model avoid generating false responses.

Learn more at https://learn.microsoft.com/en-us/azure/cognitive-services/openai/concepts/prompt-engineering#best-practices

In [24]:
# prompt_messages = [
#     {"role": "system", "content": ""},
#     {"role": "user", "content": ""}
# ]
# completion_response = perform_chat_completion(prompt_messages)
# print_messages(prompt_messages)
# print_messages([completion_response["choices"][0]["message"]])


### Example 5: Zero Shot, One Shot, Few Shot

In this example we will show how to use the `Zero Shot`, `One Shot` and `Few Shot` capabilities of the Chat Completion API. 

**Scenario 5.1: Zero Shot**

Only the command and the unsolved input is provided to the model.

| Role | Explanation |
| --- | --- |
| `system` | Suggest three names for an animal that is a superhero. |
| `user` | Dog |


**Scenario 5.2: One Shot**

One example is provided to the model before an unsolved problem is asked.

| Role | Explanation |
| --- | --- |
| `system` | Suggest three names for an animal that is a superhero. |
| `user` | Cat |
| `assistant` | Cat Damon, Pawsley Snipes and Audrey Hepurrn |
| `user` | Dog |


**Scenario 5.3: Few Shot**

Few solved examples are provided to the model before an unsolved problem is asked.

| Role | Explanation |
| --- | --- |
| `system` | Suggest three names for an animal that is a superhero. |
| `user` | Cat |
| `assistant` | Cat Damon, Pawsley Snipes and Audrey Hepurrn |
| `user` | Horse |
| `assistant` | Neigh-talie Portman, Gallop Gadot and David Hasselhoof |
| `user` | Dog |


In [54]:
box("Scenario 5.1: Generate Dog named using Zero Shot")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly

prompt_messages = [
    {"role": "system", "content": "Suggest three names for an animal that is a superhero."},
    {"role": "user", "content": "Dog"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌────────────────────────────────────────────────┐
│Scenario 5.1: Generate Dog named using Zero Shot│
└────────────────────────────────────────────────┘

[system]:
Suggest three names for an animal that is a superhero.

[user]:
Dog

[assistant]:
1. Superpup
2. Mighty Mutt
3. Captain Canine


In [55]:
box("Scenario 5.2: Generate Dog named using One Shot")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly

prompt_messages = [
    {"role": "system", "content": "Suggest three names for an animal that is a superhero."},
    {"role": "user", "content": "Cat"},
    {"role": "assistant", "content": "Cat Damon, Pawsley Snipes and Audrey Hepurrn"},
    {"role": "user", "content": "Dog"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌───────────────────────────────────────────────┐
│Scenario 5.2: Generate Dog named using One Shot│
└───────────────────────────────────────────────┘

[system]:
Suggest three names for an animal that is a superhero.

[user]:
Cat

[assistant]:
Cat Damon, Pawsley Snipes and Audrey Hepurrn

[user]:
Dog

[assistant]:
Bark Kent, Captain Canine and Wonder Woof


In [57]:
box("Scenario 5.3: Generate Dog named using Few Shot")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly

prompt_messages = [
    {"role": "system", "content": "Suggest three names for an animal that is a superhero."},
    {"role": "user", "content": "Cat"},
    {"role": "assistant", "content": "Cat Damon, Pawsley Snipes and Audrey Hepurrn"},    
    {"role": "user", "content": "Horse"},
    {"role": "assistant", "content": "Neigh-talie Portman, Gallop Gadot and David Hasselhoof"},
    {"role": "user", "content": "Dog"}
]
completion_response = perform_chat_completion(prompt_messages, type)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])



┌───────────────────────────────────────────────┐
│Scenario 5.3: Generate Dog named using Few Shot│
└───────────────────────────────────────────────┘

[system]:
Suggest three names for an animal that is a superhero.

[user]:
Cat

[assistant]:
Cat Damon, Pawsley Snipes and Audrey Hepurrn

[user]:
Horse

[assistant]:
Neigh-talie Portman, Gallop Gadot and David Hasselhoof

[user]:
Dog

[assistant]:
Bark Kent, Wonder Pug and Captain Canine


### Example 6: Exploring API parameters

The Chat Completion API has a number of parameters that can be used to control the output of the model. 

In this example we will explore the following parameters:
- `temperature` - Controls the randomness of the model. Lower values make the model more deterministic and higher values make the model more random.
- `top_p` - Controls the diversity of the model. Lower values make the model more deterministic and higher values make the model more random.

Let's explore the results we get when we make the model more deterministic and more random (or creative).

We will use the same prompt for both examples based on the last example.


In [68]:
box("Scenario 6.1: Generate Dog names more deterministically")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly

prompt_messages = [
    {"role": "system", "content": "Suggest three names for an animal that is a superhero."},
    {"role": "user", "content": "Cat"},
    {"role": "assistant", "content": "Cat Damon, Pawsley Snipes and Audrey Hepurrn"},
    {"role": "user", "content": "Dog"}
]
completion_response = perform_chat_completion(prompt_messages, type, temperature=0, top_p=0)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])
italic("Likely you'll get the same response most of the time.")



┌───────────────────────────────────────────────────────┐
│Scenario 6.1: Generate Dog names more deterministically│
└───────────────────────────────────────────────────────┘

[system]:
Suggest three names for an animal that is a superhero.

[user]:
Cat

[assistant]:
Cat Damon, Pawsley Snipes and Audrey Hepurrn

[user]:
Dog

[assistant]:
Bark Kent, Captain Canine and Wonder Woof
Likely you'll get the same response most of the time.


In [69]:
box("Scenario 6.2: Generate Dog names more randomly or creatively")
type = "azure" # change this to "openai" if you are using OpenAI's APIs directly

prompt_messages = [
    {"role": "system", "content": "Suggest three names for an animal that is a superhero."},
    {"role": "user", "content": "Cat"},
    {"role": "assistant", "content": "Cat Damon, Pawsley Snipes and Audrey Hepurrn"},
    {"role": "user", "content": "Dog"}
]
completion_response = perform_chat_completion(prompt_messages, type, temperature=2, top_p=1)
print_messages(prompt_messages)
print_messages([completion_response["choices"][0]["message"]])
italic("More likely you'll get a different response each time.")



┌────────────────────────────────────────────────────────────┐
│Scenario 6.2: Generate Dog names more randomly or creatively│
└────────────────────────────────────────────────────────────┘

[system]:
Suggest three names for an animal that is a superhero.

[user]:
Cat

[assistant]:
Cat Damon, Pawsley Snipes and Audrey Hepurrn

[user]:
Dog

[assistant]:
Ace Barkman, Blu Thunder Hounderson, Fido ElectroDasher
More likely you'll get a different response each time.


### TODO:
- Example X: Specify Output Formats
- Example X: Grounding
- Example X: Auto Validation
- Example X: Retrieval Augmented Generation
